# FastStripe Architecture

> Understanding how FastStripe works under the hood

This document explains the architectural decisions and design patterns that make FastStripe work.

## Design Philosophy

FastStripe was built with several key principles in mind:

1. **Self-documenting**: Developers should be able to discover API capabilities without external documentation
2. **Lightweight**: Minimal dependencies and overhead
3. **Consistent**: Uniform patterns across all Stripe resources
4. **Dynamic**: Automatically adapt to Stripe API changes
5. **Pythonic**: Natural Python idioms and patterns

## Core Components

### 1. Static Endpoint Definitions

FastStripe uses a static snapshot of Stripe's OpenAPI specification stored in `endpoints.py`. This file is generated from Stripe's official spec and updated using the `faststripe_update` CLI command:

In [ ]:
# Endpoints are loaded from a static file instead of fetching at runtime
from faststripe.endpoints import eps

print(f"Number of endpoints: {len(eps)}")
print(f"First endpoint: {eps[0]['path']} ({eps[0]['verb'].upper()})")

# Update to latest Stripe API version:
# $ faststripe_update

**Why this approach?**

- **Stability**: Pinned to specific Stripe API versions to prevent breaking changes
- **Faster startup**: No need to download OpenAPI spec on each import
- **Predictable behavior**: Same endpoints across environments
- **Version control**: Changes to endpoints are tracked in git
- **Easy updates**: Use `faststripe_update` command to sync with latest Stripe API

### 2. Dynamic Function Generation

Instead of hard-coding methods, FastStripe creates Python functions dynamically from the OpenAPI spec:

In [ ]:
def _mk_func(path, verb, param_info, summary, hdrs={}):
    """Create a Python function from OpenAPI endpoint definition"""
    # Create function signature with proper parameters
    sig_params = [Parameter(param['name'], Parameter.KEYWORD_ONLY, default=None) 
                  for param in param_info]
    
    # Generate docstring from API documentation
    param_docs = '\n'.join(f"    {param['name']}: {param['description']}" 
                          for param in param_info)
    docstring = f"{summary}\n\nParameters:\n{param_docs}" if param_docs else summary
    
    # Create the actual function
    def m(**kwargs): 
        return dict2obj(getattr(httpx, verb)(stripe_api_url + path, 
                                           headers=hdrs,
                                           params=_flatten_data(kwargs)).json())
    
    # Attach signature and documentation
    m.__signature__ = Signature(sig_params)
    m.__doc__ = docstring
    return m

**Benefits:**

- IDE auto-completion and parameter hints
- Consistent function signatures
- Built-in documentation
- Reduced code maintenance

### 3. Resource Organization

Endpoints are grouped by resource type based on their `operationId`:

In [ ]:
def _parse_operation_id(op_id):
    """Convert PostCustomers -> ('customers', 'create')"""
    parts = re.findall(r'[A-Z][a-z]*', op_id)
    verb, *resource_parts = [p.lower() for p in parts]
    resource = '_'.join(resource_parts) if resource_parts else 'misc'
    method_name = 'create' if verb == 'post' else 'fetch' if verb == 'get' else verb
    return resource, method_name

# Examples:
# PostCustomers -> ('customers', 'create')
# GetCustomers -> ('customers', 'fetch')
# PostCheckoutSessions -> ('checkout_sessions', 'create')

This creates intuitive groupings:
```python
sapi.customers.create()    # POST /v1/customers
sapi.customers.fetch()     # GET /v1/customers
sapi.products.create()     # POST /v1/products
sapi.checkout_sessions.create()  # POST /v1/checkout/sessions
```

### 4. Data Flattening

Stripe expects form-encoded data with flattened parameters. FastStripe handles this automatically:

In [ ]:
def _flatten_data(data, prefix=''):
    """Convert nested dictionaries to Stripe's flat format"""
    result = {}
    for k, v in data.items():
        key = f'{prefix}[{k}]' if prefix else k
        if isinstance(v, dict): 
            result.update(_flatten_data(v, key))
        elif isinstance(v, list): 
            for i, item in enumerate(v):
                if isinstance(item, dict): 
                    result.update(_flatten_data(item, f'{key}[{i}]'))
                else: 
                    result[f'{key}[{i}]'] = item
        else: 
            result[key] = v
    return result

# Example transformation:
# {
#     'address': {
#         'line1': '123 Main St',
#         'city': 'San Francisco'
#     }
# }
# Becomes:
# {
#     'address[line1]': '123 Main St',
#     'address[city]': 'San Francisco'
# }

### 5. Response Enhancement

API responses are converted to AttrDict objects for convenient access:

In [ ]:
# Standard dictionary access
customer['email']           # Works but verbose

# AttrDict allows dot notation
customer.email              # More Pythonic
customer.address.city       # Nested access
customer.metadata.user_id   # Custom metadata

## High-Level API Design

The convenience methods (`one_time_payment`, `subscription`) follow a pattern:

1. **Create or find products/prices** using helper methods
2. **Compose complex requests** from simple parameters
3. **Return meaningful objects** with accessible properties

This abstracts away the complexity of Stripe's multi-step workflows while maintaining flexibility.